# Importing libraries

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from category_encoders import TargetEncoder

from sklearn.preprocessing import MinMaxScaler

from sklearn.decomposition import PCA
from sklearn.feature_selection import chi2

from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE

from sklearn.svm import SVC
from sklearn import tree
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score, accuracy_score, roc_auc_score, fbeta_score

# Reading file and tidying

In [3]:
df = pd.read_csv("loanprediction.csv")
df.drop("Id", axis=1, inplace=True)
df.columns = ["income", "age", "experience", "marital_status", "house_ownership", "car_ownership", "profession", "city", "state", "current_job_years", "current_house_years", "risk_flag"]
numerical_cols = ["income", "age", "experience", "current_job_years", "current_house_years"]
#df.info()

# Feature selection
H0 :- There is no relationship between categorical feature and target variable\
H1 :- There is some relationship between categorical feature and target variable\
If p-value ≥0.05, the null hypothesis is not rejected and there is no any relationship between target variable and categorical features.

## Chi-Square

In [4]:
# # https://medium.com/analytics-vidhya/categorical-feature-selection-using-chi-squared-test-e4c0d0af6b7e
# # https://towardsdatascience.com/using-the-chi-squared-test-for-feature-selection-with-implementation-b15a4dad93f1

# # Standardisation gives negative values, chi-square does not allow negative values

# chi_scores = chi2(x,y)
# p_values = pd.Series(chi_scores[1],index = x.columns)
# p_values.sort_values(ascending = False , inplace = True)
# p_values.plot.bar()

# # conclusion: current_house_years, profession_encoded, state_encoded have no relationship with target variable
# x = x.drop(["current_house_years", "profession_encoded", "state_encoded"], axis=1)

## PCA

In [5]:
# pca_loan = PCA()
# x_train = pca_loan.fit_transform(x_train)
# x_test = pca_loan.fit_transform(x_test)

# Train test split

In [6]:
y = df["risk_flag"]
x = df.drop("risk_flag", axis=1)

In [7]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 2021,stratify=y)

# Data preprocessing - Encoding categorical columns and scaling numerical columns

In [8]:
def data_preprocessing(df_x, df_y):
    
    # Label encoding categorical columns with 2 types of categories
    x = df_x.copy()
    label_enc = LabelEncoder()
    x["marital_status"] = label_enc.fit_transform(x["marital_status"])
    x["car_ownership"] = label_enc.fit_transform(x["car_ownership"])
    
    # One Hot Encoding house_ownership column & Combining back to dataframe
    onehot_encoder = OneHotEncoder(handle_unknown='ignore', sparse=False)
    
    house_ownership_values = onehot_encoder.fit_transform( x[['house_ownership']] )
    house_ownership_labels = np.array(["norent_noown", "owned", "rented"]).ravel()
    house_ownership_df = pd.DataFrame(house_ownership_values, columns=house_ownership_labels)

    x.reset_index(drop=True, inplace=True)
    house_ownership_df.reset_index(drop=True, inplace=True)
    x = pd.concat([ x, house_ownership_df], axis=1)
    
    x.drop("house_ownership", axis=1, inplace=True)
    
    # Target Encoding the high cardinality categorical columns: profession, city, state
    # https://medium.com/analytics-vidhya/target-encoding-vs-one-hot-encoding-with-simple-examples-276a7e7b3e64
    profession_target_enc = TargetEncoder()
    x["profession_encoded"] = profession_target_enc.fit_transform(x["profession"], df_y)
    city_target_enc = TargetEncoder()
    x["city_encoded"] = city_target_enc.fit_transform(x["city"], df_y)
    state_target_enc = TargetEncoder()
    x["state_encoded"] = state_target_enc.fit_transform(x["state"], df_y)
    x.drop("profession", axis=1, inplace=True)
    x.drop("city", axis=1, inplace=True)
    x.drop("state", axis=1, inplace=True)
    
    # https://stackoverflow.com/questions/51237635/difference-between-standard-scaler-and-minmaxscaler
    # https://www.geeksforgeeks.org/standardscaler-minmaxscaler-and-robustscaler-techniques-ml/
    min_max_scaler = MinMaxScaler()
    x[numerical_cols] = min_max_scaler.fit_transform(x[numerical_cols])
    # need to scale the encoded columns?
    
    return x

In [9]:
x_train = data_preprocessing(x_train, y_train)
x_test = data_preprocessing(x_test, y_test)

In [10]:
def run_variation_model(x_train, y_train, x_test, y_test, variation):
    if "pca" in variation:
        pca = PCA(0.9, random_state=2021)
        x_train = pca.fit_transform(x_train)
        x_test = pca.fit_transform(x_test)
        
    if "chi_square" in variation:
        chi_scores = chi2(x_train, y_train)
        p_values = pd.Series(chi_scores[1], index = x_train.columns)
        p_values.sort_values(ascending = False , inplace = True)
        p_values.plot.bar()
        x_train = x_train.drop(["profession_encoded", "state_encoded", "city_encoded", "income"], axis=1)
        x_test = x_test.drop(["profession_encoded", "state_encoded", "city_encoded", "income"], axis=1)
        
    if "smote" in variation:
        oversampler = SMOTE(random_state=2021)
        x_train, y_train = oversampler.fit_resample(x_train, y_train)
        
    classifier.fit(x_train, y_train)
    y_pred = classifier.predict(x_test)
    
    print(f"-------------------------TEST SCORES for {variation}-----------------------")
    print(f"Recall: {recall_score(y_test, y_pred)}")
    print(f"Precision: {precision_score(y_test, y_pred)}")
    print(f"F2-Score: {fbeta_score(y_test, y_pred, beta=2)}")
    print(f"Accuracy score: {accuracy_score(y_test, y_pred)}")
    print(f"AUC Score: {roc_auc_score(y_test, y_pred)}")
    print()

#### Applying dtree model

In [ ]:
#dtree = tree.DecisionTreeClassifier(random_state=69)

# dtree.fit(x_train, y_train)
# x_test = data_preprocessing(x_test, y_test)
# #x_test = pca_loan.fit_transform(x_test)
# y_pred = dtree.predict(x_test)
# cm = confusion_matrix(y_test, y_pred)
# print(cm)

classifier = SVC(kernel = 'rbf', random_state = 0)


variations = ["base", "smote", "chi_square", "pca", "smote, chi_square", "smote, pca"]

for variation in variations:
    run_variation_model(x_train, y_train, x_test, y_test, variation)

-------------------------TEST SCORES for base-----------------------
Recall: 0.0
Precision: 0.0
F2-Score: 0.0
Accuracy score: 0.8770039682539682
AUC Score: 0.5



C:\Users\thine\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
started at 2:18pm